In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
!pip install tqdm --upgrade

In [ ]:
!pip install lorem

In [ ]:
!pip install google_images_download

In [ ]:
from openpyxl import load_workbook
from tqdm.notebook import tqdm
from copy import deepcopy
from arrow import Arrow
from datetime import datetime, timedelta
from random import choice
from django_bulk_update.helper import bulk_update
from simple_history.utils import bulk_create_with_history
from IPython.display import display, Markdown
import lorem
import json
import os
import random
import copy
import math

In [ ]:
from fixtures.mock_empreiteira.permission_names import pretty_print_permission
from helpers.validators.brazilian_documents import generate_cnpj
from helpers.strings import get_autonumber_array, clean_latin_string
from helpers.signals import DisableSignals
from helpers.apps.services import (
    create_or_update_services_and_usages,
    create_services_from_measurement,
)
from apps.reportings.signals import reporting_create, auto_add_reporting_number
from django_bulk_update.helper import bulk_update

# CONCESSIONARIA

In [ ]:
company_name = "Demo Concessionárias - 10/2020"
prefixo_folder = "fixtures/mock_concessionaria/"
prefixo_users = "demo."
admin_user_count = 4
campo_user_count = 25

In [ ]:
companies = Company.objects.filter(name=company_name)

In [ ]:
company = companies.first()

## WARNING! The following two cells will delete an existing company with the same name

In [ ]:
User.objects.filter(companies__in=companies).delete()

In [ ]:
companies.delete()

## Define permission levels, users and firms

In [ ]:
user_firm_count = {
    "Inativo": {
        "permission": None,
        "user_count": 0,
        "firm_names": [],
        "firms": [],
        "users": []
    },
    "Gerente": {
        "permission": None,
        "user_count": 3,
        "firm_names": ["Gerência"],
        "firms": [],
        "users": []
    },
    "Coordenador": {
        "permission": None,
        "user_count": 3,
        "firm_names": ["Coordenação"],
        "firms": [],
        "users": []
    },
    "Engenheiro": {
        "permission": None,
        "user_count": 5,
        "firm_names": ["Engenharia"],
        "firms": [],
        "users": []
    },
    "Inspetor": {
        "permission": None,
        "user_count": 20,
        "firm_names": ["Inspeção Lote 1", "Inspeção Lote 2"],
        "firms": [],
        "users": []
    },
    "Terceiro": {
        "permission": None,
        "user_count": 30,
        "firm_names": ["MaxxEmpreiteira", "RodoConstruções", "Silva Mão de Obra", "Empreiteira Carneiro"],
        "firms": [],
        "users": []
    },
    "ARTESP": {
        "permission": None,
        "user_count": 5,
        "firm_names": ["ARTESP"],
        "firms": [],
        "users": []
    },
    "Sala técnica": {
        "permission": None,
        "user_count": 5,
        "firm_names": ["Sala Técnica"],
        "firms": [],
        "users": []
    }
}

## User

Create users before creating the Company because we need somebody to be the `owner` of the Company

In [ ]:
with open(prefixo_folder + 'first_names.json', 'r+') as fo:
    first_names = json.load(fo)
    
with open(prefixo_folder + 'last_names.json', 'r+') as fo:
    last_names = json.load(fo)
    
def get_user_params():
    first = choice(first_names)
    last = choice(last_names)
    username = prefixo_users + first.lower() + "." + last.lower()
    return first, last, username

In [ ]:
for _, user_firms in user_firm_count.items():

    first, last, username = get_user_params()
    for _ in range(user_firms['user_count']):
        while User.objects.filter(username=username):
            first, last, username = get_user_params()

        user = User.objects.create(
            first_name=first,
            last_name=last,
            username=username,
        )
        user.set_password('kartado')
        user.save()
        user_firms['users'].append(user)

In [ ]:
user_firm_count['Sala técnica']['users']

## Company

Create a Company. The name will be provided when the script runs.

In [ ]:
with open(prefixo_folder + 'metadata.json', 'r+') as fo:
    metadata = json.load(fo)

In [ ]:
with open(prefixo_folder + 'custom_options.json', 'r+') as fo:
    custom_options = json.load(fo)

In [ ]:
company = Company.objects.create(
    name=company_name,
    metadata=metadata,
    custom_options=custom_options,
    active=True,
    cnpj=generate_cnpj(),
    owner=user_firm_count['Gerente']['users'][0]
)

## UserPermission

Create permission levels according to object above

In [ ]:
for permission_name in user_firm_count.keys():
    with open(prefixo_folder + 'permissions/{}.json'.format(permission_name), 'r+') as fo:
        permission_json = json.load(fo)
        permission_obj = UserPermission.objects.create(
            name=permission_name,
            permissions=permission_json
        )
        permission_obj.companies.set([company])
        user_firm_count[permission_name]['permission'] = permission_obj

## UserInCompany

Associate all the created users

In [ ]:
for user_firms in user_firm_count.values():
    for user in user_firms['users']:        
        user_in_company = UserInCompany.objects.create(
            user=user,
            company=company,
            permissions=user_firms['permission']
        )

## CompanyGroup

Create a CompanyGroup with the same name as the company.

Set the CompanyGroup of the Company and all the users to be the newly created one.

In [ ]:
company_group = CompanyGroup.objects.create(
    name=company_name,
    key_user=user_firm_count['Gerente']['users'][0]
)

In [ ]:
company.company_group = company_group
company.save()

for user_firms in user_firm_count.values():
    for user in user_firms['users']:        
        user.company_group = company_group
        user.save()

## Firm

Create one object for admins and several for the other users

In [ ]:
Firm.objects.filter(company=company).delete()

In [ ]:
for user_firms in user_firm_count.values():
    for firm in user_firms['firm_names']:
        firm_obj = Firm.objects.create(
            name=firm,
            company=company,
            cnpj=generate_cnpj(),
            is_company_team= (not user_firms['permission'].name == "Terceiro"),
            members_amount = math.ceil(user_firms['user_count'] / len(user_firms['firm_names']))
        )
        user_firms['firms'].append(firm_obj)
    
    for user in user_firms['users']:
        chosen_firm = choice(user_firms['firms'])
        user_in_firm = UserInFirm.objects.create(
            user=user,
            firm=chosen_firm
        )
        if not chosen_firm.manager:
            chosen_firm.manager = user
            chosen_firm.save()

## TileLayers

Create using this objects below and tie all of them to the newly created company.

In [ ]:
tile_objects = [{"type": "mapbox", "description": "Satélite", "name": "Satélite", "provider_info": {"url": "https://api.mapbox.com/styles/v1/natank/cjmmhqxpt0jsn2smsihy5nx5w/tiles/{z}/{x}/{y}.mapbox", "type": "mapbox", "order": 3, "accessToken": "pk.eyJ1IjoibmF0YW5rIiwiYSI6ImNqbW1oNngyYjBoZWEzcm83dDRnZTRmMnoifQ.IJHj30XcphWWzx7Luvc-2A", "attribution": "Licensed by &copy; <a href=\"https://www.mapbox.com/\">Mapbox</a>", "styleString": "mapbox://styles/natank/ck0z9h5s014wj1cqum4jjssbk"}},
{"type": "mapbox", "description": "Mapa Claro", "name": "Mapa Claro", "provider_info": {"url": "https://api.mapbox.com/styles/v1/natank/cjmmhqxpt0jsn2smsihy5nx5w/tiles/{z}/{x}/{y}.mapbox", "type": "mapbox", "order": 4, "accessToken": "pk.eyJ1IjoibmF0YW5rIiwiYSI6ImNqbW1oNngyYjBoZWEzcm83dDRnZTRmMnoifQ.IJHj30XcphWWzx7Luvc-2A", "attribution": "Licensed by &copy; <a href=\"https://www.mapbox.com/\">Mapbox</a>", "styleString": "mapbox://styles/natank/ck0z9bp0r0z4l1cqhp4wt15ew"}},
{"type": "mapbox", "description": "Mapa Escuro", "name": "Mapa Escuro", "provider_info": {"url": "https://api.mapbox.com/styles/v1/natank/cjmmhqxpt0jsn2smsihy5nx5w/tiles/{z}/{x}/{y}.mapbox", "type": "mapbox", "order": 5, "accessToken": "pk.eyJ1IjoibmF0YW5rIiwiYSI6ImNqbW1oNngyYjBoZWEzcm83dDRnZTRmMnoifQ.IJHj30XcphWWzx7Luvc-2A", "attribution": "Licensed by &copy; <a href=\"https://www.mapbox.com/\">Mapbox</{a>", "styleString": "mapbox://styles/natank/ck0z9os2914au1cqqaj464p5m"}},
{"type": "openStreetMap", "description": "OpenStreetMap", "name": "OpenStreetMap", "provider_info": {"url": "https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png", "type": "openStreetMap", "order": 6, "attribution": "&amp;copy <a href=\"http://osm.org/copyright\">OpenStreetMap</a> contributors"}},
{"type": "mapbox", "description": "Relevo", "name": "Relevo", "provider_info": {"url": "https://api.mapbox.com/styles/v1/natank/cjmmhqxpt0jsn2smsihy5nx5w/tiles/{z}/{x}/{y}.mapbox", "type": "mapbox", "order": 2, "accessToken": "pk.eyJ1IjoibmF0YW5rIiwiYSI6ImNqbW1oNngyYjBoZWEzcm83dDRnZTRmMnoifQ.IJHj30XcphWWzx7Luvc-2A", "attribution": "Licensed by &copy; <a href=\"https://www.mapbox.com/\">Mapbox</a>", "styleString": "mapbox://styles/natank/ck0z7y0bi0g6x1cq1oxwo8bhd"}},
{"type": "mapbox", "description": "Padrão", "name": "Padrão", "provider_info": {"url": "https://api.mapbox.com/styles/v1/natank/cjmmhqxpt0jsn2smsihy5nx5w/tiles/{z}/{x}/{y}.mapbox", "type": "mapbox", "order": 1, "accessToken": "pk.eyJ1IjoibmF0YW5rIiwiYSI6ImNqbW1oNngyYjBoZWEzcm83dDRnZTRmMnoifQ.IJHj30XcphWWzx7Luvc-2A", "attribution": "Licensed by &copy; <a href=\"https://www.mapbox.com/\">Mapbox</a>", "styleString": "mapbox://styles/natank/cjn50jrc82oyq2rlew8h2c7yp"}}]

In [ ]:
for item in tile_objects:
    tile_layer = TileLayer.objects.create(**item)
    tile_layer.companies.add(company)
    tile_layer.save()

## ShapeFile

not necessary for now

## OccurrenceType and OccurrenceTypeSpecs

Create objects tying the OccurrenceType objects to the new Company. Randomize the color.

In [ ]:
class DateTimeAwareJSONDecoder(json.JSONDecoder):
    """ 
    Converts a json string, where datetime and timedelta objects were converted
    into objects using the DateTimeAwareJSONEncoder, back into a python object.
    """

    def __init__(self):
        json.JSONDecoder.__init__(self, object_hook=self.dict_to_object)

    def dict_to_object(self, d):
        if '__type__' not in d:
            return d

        type = d.pop('__type__')
        if type == 'datetime':
            return datetime(**d)
        elif type == 'timedelta':
            return timedelta(**d)
        else:
            # Oops... better put this back together.
            d['__type__'] = type
            return d

In [ ]:
new_forms = []

forms_folder = prefixo_folder + "formularios"
for index, file in enumerate(os.listdir(forms_folder)):
    if not '.json' in file:
        continue
    file_name = '{}/{}'.format(forms_folder, file)
    print('------')
    print(file_name)
    with open(file_name, 'r+') as fo:
        decoder = DateTimeAwareJSONDecoder()
        form = json.load(fo)
        form['deadline'] = decoder.decode(form['deadline'])
        print(form['displayName'])
        print(form['deadline'])
        new_forms.append(form)

In [ ]:
occurrence_types = []
for form in new_forms:
    form_copy = deepcopy(form)
    if 'deadline' in form_copy.keys():
        del form_copy['deadline']
    if 'kind' in form_copy.keys():
        del form_copy['kind']
    occurrence_type = OccurrenceType(
        name=form['displayName'],
        occurrence_kind=form['kind'],
        form_fields = form_copy,
        created_by=user_firm_count['Gerente']['users'][0],
        deadline= form['deadline'] if 'deadline' in form else None
    )
    occurrence_types.append(occurrence_type)

In [ ]:
for occurrence_type in occurrence_types:
    occurrence_type.save()
    spec = OccurrenceTypeSpecs(
        occurrence_type = occurrence_type,
        company = company,
        color = "#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
    )
    spec.save()

In [ ]:
def get_color():
    return "#%06x" % random.randint(0, 0xFFFFFF)

## ServiceOrderActionStatus and ServiceOrderActionStatusSpecs

Create copies of the ones from Arteris Litoral Sul

Create one for each status

In [ ]:
statuses = [{"name": "Identificado", "color": "#ffeb3b", "order": 1},
{"name": "Programado", "color": "#f44336", "order": 2},
{"name": "Emergencial", "color": "#f57c00", "order": 3},
{"name": "Em Análise", "color": "#9c27b0", "order": 4},
{"name": "Pré-Programação", "color": "#e91e63", "order": 5},
{"name": "Cancelado", "color": "#d9d9d9", "order": 6},
{"name": "Justificado", "color": "#2196f3", "order": 7},
{"name": "Executado", "color": "#4caf50", "order": 8}]

In [ ]:
for status in statuses:    
    status_obj = ServiceOrderActionStatus.objects.create(
        name=status['name'],
        kind="REPORTING_STATUS"
    )
    status_specs = ServiceOrderActionStatusSpecs.objects.create(
        company=company,
        status=status_obj,
        color=status['color'],
        order=status['order']
    )

## ApprovalFlow for Reporting

In [ ]:
ApprovalFlow.objects.filter(company=company, target_model="reportings.Reporting").delete()

In [ ]:
for permission in UserPermission.objects.filter(companies=company).exclude(name="Inativo"):
    permission.permissions['Reporting']['can_approve'] = True
    permission.save()

In [ ]:
flow = ApprovalFlow(
    name="Fluxo Apontamentos",
    target_model="reportings.Reporting",
    company=company
)
flow.save()

In [ ]:
elaboracao = ApprovalStep(
    name="Em Elaboração",
    approval_flow=flow,
)
elaboracao.responsible_created_by = True
elaboracao.save()
elaboracao.responsible_firms.add(Firm.objects.get(name="Engenharia", company=company))
elaboracao.responsible_firms.add(Firm.objects.get(name="Sala Técnica", company=company))

In [ ]:
homolog_normal = ApprovalStep(
    name="Aguardando Aprovação",
    approval_flow=flow,
)
homolog_normal.save()
homolog_normal.responsible_firms.add(Firm.objects.get(name="Engenharia", company=company))
homolog_normal.responsible_firms.add(Firm.objects.get(name="Sala Técnica", company=company))

In [ ]:
revisao = ApprovalStep(
    name="Necessita Revisão",
    approval_flow=flow,
)
revisao.responsible_created_by = True
revisao.save()
revisao.responsible_firms.add(Firm.objects.get(name="Engenharia", company=company))
revisao.responsible_firms.add(Firm.objects.get(name="Sala Técnica", company=company))

In [ ]:
deferido = ApprovalStep(
    name="Aprovado",
    approval_flow=flow,
)
deferido.save()

In [ ]:
indeferido = ApprovalStep(
    name="Reprovado",
    approval_flow=flow,
)
indeferido.save()

In [ ]:
trans_homolog_normal = ApprovalTransition(
    name="Solicitar Aprovação",
    origin=elaboracao,
    destination=homolog_normal,
    callback={
        "change_fields": [
            {"name": "approved", "value": True}, 
            {"name": "done", "value": True}
        ]
    },
    condition={
        "===": [{
            "var": "request.action"
            }, "Solicitar Aprovação"]
    },
    button={"icon": "assignment", "color": "primary", "variant": "raised"}
)
trans_homolog_normal.save()


In [ ]:
trans_homolog_again_normal = ApprovalTransition(
    name="Solicitar Aprovação",
    origin=revisao,
    destination=homolog_normal,
    condition={
        "===": [{
            "var": "request.action"
        }, "Solicitar Aprovação"]
    },
    button={"icon": "assignment", "color": "primary", "variant": "raised"}
)
trans_homolog_again_normal.save()


In [ ]:
trans_revisar_normal = ApprovalTransition(
    name="Solicitar Revisão",
    origin=homolog_normal,
    destination=revisao,
    condition={
        "===": [{
            "var": "request.action"
        }, "Solicitar Revisão"]
    },
    button={"icon": "warning", "color": "amber", "variant": "outlined", "needs_dialog": True}
)
trans_revisar_normal.save()


In [ ]:
trans_homologar_normal = ApprovalTransition(
    name="Aprovar",
    origin=homolog_normal,
    destination=deferido,
    condition={
        "===": [{
            "var": "request.action"
        }, "Aprovar"] 
    },
    button={"icon": "check", "color": "primary", "variant": "raised"}
)
trans_homologar_normal.save()


In [ ]:
trans_rejeitar_normal = ApprovalTransition(
    name="Reprovar",
    origin=homolog_normal,
    destination=indeferido,
    condition={
        "===": [{
            "var": "request.action"
        }, "Reprovar"]
    },
    button={"icon": "clear", "color": "red", "variant": "outlined"}
)
trans_rejeitar_normal.save()


In [ ]:
company.metadata['approved_approval_steps'] = [str(deferido.uuid)]

In [ ]:
company.save()

## ApprovalFlow for MeasurementBulletin

In [ ]:
ApprovalFlow.objects.filter(company=company, target_model="service_orders.MeasurementBulletin").delete()

In [ ]:
flow = ApprovalFlow(
    name="Fluxo Boletins de Medição",
    target_model="service_orders.MeasurementBulletin",
    company=company
)
flow.save()

In [ ]:
step1 = ApprovalStep(
    name="Aguardando Aprovação",
    approval_flow=flow,
    field_options={"required": ["relationships.permissions.data.id"]}
)
step1.save()
for firm in Firm.objects.filter(company=company):
    step1.responsible_users.add(firm.manager)
step1.responsible_firms.add(Firm.objects.get(name="Engenharia", company=company))
step1.responsible_firms.add(Firm.objects.get(name="Sala Técnica", company=company))

In [ ]:
step2 = ApprovalStep(
    name="Negado",
    approval_flow=flow,
    field_options={"required": ["relationships.permissions.data.id"]}
)
step2.save()
for firm in Firm.objects.filter(company=company):
    step2.responsible_users.add(firm.manager)
step2.responsible_firms.add(Firm.objects.get(name="Engenharia", company=company))
step2.responsible_firms.add(Firm.objects.get(name="Sala Técnica", company=company))

In [ ]:
step3 = ApprovalStep(
    name="Aprovado",
    approval_flow=flow,
    field_options={"required": ["relationships.permissions.data.id"]}
)
step3.save()
for firm in Firm.objects.filter(company=company):
    step3.responsible_users.add(firm.manager)
step3.responsible_firms.add(Firm.objects.get(name="Engenharia", company=company))
step3.responsible_firms.add(Firm.objects.get(name="Sala Técnica", company=company))

In [ ]:
transition1 = ApprovalTransition(
    name="Aprovar",
    origin=step1,
    destination=step3,
    condition={"===": [{"var": "request.action"}, "Aprovar"]},
    callback={},
    button={"icon": "check", "color": "primary", "variant": "raised"}
)
transition1.save()

In [ ]:
transition2 = ApprovalTransition(
    name="Rejeitar",
    origin=step1,
    destination=step2,
    condition={"===": [{"var": "request.action"}, "Rejeitar"]},
    callback={},
    button={"icon": "clear", "color": "red", "variant": "outlined"}
)
transition2.save()

## ApprovalFlow for AccessRequest

In [ ]:
flow = ApprovalFlow(
    name="Fluxo Usuários",
    target_model="companies.AccessRequest",
    company=company
)
flow.save()

In [ ]:
step1 = ApprovalStep(
    name="Aguardando Aprovação",
    approval_flow=flow,
    field_options={"required": ["relationships.permissions.data.id"]},
    auto_execute_transition=True
)
step1.save()

In [ ]:
step3 = ApprovalStep(
    name="Aprovado",
    approval_flow=flow,
    field_options={"required": ["relationships.permissions.data.id"]}
)
step3.save()

In [ ]:
transition1 = ApprovalTransition(
    name="Aprovar",
    origin=step1,
    destination=step3,
    condition={"===": [{"var": "request.action"}, "Aprovar"]},
    callback={"change_fields": [{"name": "approved", "value": True}, {"name": "done", "value": True}], "create_user_in_company": True},
)
transition1.save()

## Road

Create a single demo road containing a part of BR282-SC

In [ ]:
road = Road(
    marks = {"0": {"km": 114, "key": 0, "index": 0, "point": {"type": "Point", "coordinates": [-49.3880981, -27.7483643]}}, "1": {"km": 223, "key": 1, "index": 745, "point": {"type": "Point", "coordinates": [-50.363131, -27.800003]}}, "2": {"km": 223.001, "key": 2, "index": 745, "point": {"type": "Point", "coordinates": [-50.363131, -27.800003]}}},
    direction = 0,
    name = "BR282-SC-DEMO",
    uf = "42",
)
road.save()
road.company.add(company)

## Reporting

Generate 10000 new objects according to the instructions below:

```
number: autogenerate
company: newly created company
road_name: pick a random one
road: same as above
km: random within the range of the selected road
point: let the algorithm calculate
direction: pick a random one according to whats available in the company custom_options
lane: pick a random one according to whats available in the company custom_options
created_by: one of the newly created users
firm: the firm of the user
occurrence_type: pick a random one
form_data: randomly fill according to the form_fields of the selected occurrence_type
executed_at: random between None and a date in the past year
created_at: a random date in the past year. If executed_at was filled, make sure this date is before executed_at
found_at: a random date in the past year. If executed_at was filled, make sure this date is before executed_at
updated_at: a random date in the past year. If executed_at was filled, make sure this date is before executed_at
status: pick a random one
```

In [ ]:
from datetime import date, timedelta, datetime
from django.utils import timezone
# from pytz import timezone as tz

def get_dates(firm, user):
    job = None
    executed_at = None
    
    today_date = timezone.now()
    start_date = today_date.toordinal()
    end_date = today_date.replace(month=today_date.month-3).toordinal()
    rand = date.fromordinal(random.randint(end_date, start_date))
    dt = datetime.combine(rand, datetime.min.time())
    created_at = (dt + timedelta(hours=5)).replace(tzinfo=today_date.tzinfo)
    found_at = (created_at - timedelta(hours=random.randint(0, 23))).replace(hour = random.randint(7, 19))
    updated_at = (created_at + timedelta(days=random.randint(0, 7))).replace(hour = random.randint(7, 19))
    executed = (created_at + timedelta(days=random.randint(0, 7))).replace(hour = random.randint(7, 19))
    
    jobs = Job.objects.filter(
        company=company,
        start_date__lte=found_at,
        end_date__gte=found_at,
    )
    with DisableSignals():
        if not jobs:
            jobs = [
                Job.objects.create(
                    company=company,
                    start_date=found_at + timedelta(days=2),
                    end_date=found_at + timedelta(days=20),
                    firm=firm,
                    created_by=user,
                    title=lorem.sentence(),
                    worker=user
                )
            ]

    if today_date - found_at > timedelta(days=30):
        status = status_executado
    else:
        status = choice(ServiceOrderActionStatus.objects.filter(companies=company))
    
    if status == status_executado:
        executed_at = executed
        job = choice(jobs)
    elif status == status_programado:
        job = choice(jobs)
        
    
    return created_at, found_at, updated_at, executed_at, job, status

In [ ]:
import re

def snake_case(name):
    s1 = re.sub("(.)([A-Z][a-z]+)", r"\1_\2", name)
    return re.sub("([a-z0-9])([A-Z])", r"\1_\2", s1).lower()

def snake_dict(in_dict):
    out_dict = {}
    for key, value in in_dict.items():
        out_dict[snake_case(key)] = value
    return out_dict

In [ ]:
def get_form_data(occ_type):
    form_data = {}
    if occ_type.form_fields:
        if "fields" in occ_type.form_fields:
            fields = occ_type.form_fields["fields"]
            for in_field in fields:
                field = snake_dict(in_field)
                if "api_name" in field and "data_type" in field:
                    if "code" in field['api_name'].lower():
                        abc = random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 0])
                        letter = chr(ord('A')+abc)
                        plate = letter * 3 + str(abc) * 4
                        form_data[field["api_name"]] = plate
                    elif field["data_type"] == "float":
                        if field["api_name"] == "width":
                            form_data[field["api_name"]] = random.choice([3.8, 7.6])
                        elif field["api_name"] == "length":
                            if occ_type.occurrence_kind == "1":
                                form_data[field["api_name"]] = round(random.uniform(1, 5), 2)
                            else:
                                form_data[field["api_name"]] = round(random.uniform(5, 50), 2)
                        elif field["api_name"] == "height":
                            form_data[field["api_name"]] = round(random.uniform(0.01, 0.2), 2)
                        elif field["api_name"] == "rain":
                            form_data[field["api_name"]] = round(random.random() * 10, 2)
                        else:
                            form_data[field["api_name"]] = round(random.uniform(0, 1), 2)
                    elif field["data_type"] == "string":
                        form_data[field["api_name"]] = " ".join(lorem.sentence().split(" ")[0:2])
                    elif field["data_type"] == "number":
                        form_data[field["api_name"]] = int(random.uniform(0, 1)*10)
                    elif field["data_type"] == "textArea":
                        form_data[field["api_name"]] = lorem.sentence()
                    elif field["data_type"] == "boolean":
                        form_data[field["api_name"]] = True
                    elif field["data_type"] == "select":
                        if "select_options" in field:
                            if "options" in field["select_options"]:
                                value = field["select_options"]["options"][0]["value"]
                                form_data[field["api_name"]] = value
                    elif field["data_type"] == "selectMultiple":
                        if "select_options" in field:
                            if "options" in field["select_options"]:
                                value = field["select_options"]["options"][0]["value"]
                                form_data[field["api_name"]] = [value]

    return form_data

In [ ]:
direction_options = custom_options["reporting"]["fields"]["direction"]["selectOptions"]["options"]
directions = [item["value"] for item in direction_options]

lane_options = custom_options["reporting"]["fields"]["lane"]["selectOptions"]["options"]
lanes = [item["value"] for item in lane_options]

In [ ]:
all_types = list(OccurrenceType.objects.filter(company=company).exclude(occurrence_kind__in=["5", "6", "15", "16"]))
random.shuffle(all_types)

In [ ]:
roads = Road.objects.filter(company=company)

In [ ]:
firms_campo = user_firm_count['Terceiro']['firms']

In [ ]:
status_executado = ServiceOrderActionStatus.objects.get(companies=company, name="Executado")

In [ ]:
status_programado = ServiceOrderActionStatus.objects.get(companies=company, name="Programado")

In [ ]:
Reporting.objects.filter(company=company).delete()

In [ ]:
Job.objects.filter(company=company).delete()

In [ ]:
not_executed_approval_step = ApprovalStep.objects.get(
    name="Em Elaboração", 
    approval_flow__company=company, 
    approval_flow__target_model="reportings.Reporting"
)
approved_approval_step = ApprovalStep.objects.get(
    approval_flow__company=company,
    approval_flow__target_model="reportings.Reporting",
    name="Aprovado"
)
other_approval_steps = ApprovalStep.objects.filter(
    approval_flow__company=company,
    approval_flow__target_model="reportings.Reporting"
).exclude(uuid__in=[approved_approval_step.uuid, not_executed_approval_step.uuid])

executed_approval_steps = 10 * [approved_approval_step] + list(other_approval_steps)

for _ in tqdm(range(30)):
    reportings_to_create = []
    for i in range(100):
        road = choice(roads)
        kms = [road.marks[item]["km"] for item in road.marks]
        firm = choice(firms_campo)
        occ_type = choice(all_types)
        created, found, updated, executed, job, status = get_dates(firm, firm.manager)

        with DisableSignals():
            rep = Reporting(
                company=company,
                road_name=road.name,
                road=road,
                km=round(random.uniform(min(kms), max(kms)), 2),
                direction=choice(directions),
                lane=choice(lanes),
                firm=firm,
                created_by=firm.manager,
                occurrence_type=occ_type,
                form_data=get_form_data(occ_type),
                status=status,
                executed_at=executed,
                created_at=created,
                updated_at=updated,
                due_at=created + occ_type.deadline if occ_type.deadline else created + timedelta(days=7),
                found_at=found,
                job=job,
                approval_step = not_executed_approval_step if not executed else choice(executed_approval_steps)
            )
            auto_add_reporting_number(Reporting, rep)
            rep._state.adding = True
            reporting_create(Reporting, rep)
            reportings_to_create.append(rep)
            
    bulk_create_with_history(reportings_to_create, Reporting)

In [ ]:
Reporting.objects.filter(road__isnull=True, company=company).delete()

In [ ]:
def add_reporting_number(instance):
    if instance.occurrence_type:
        try:
            occurrence_kind = instance.occurrence_type.occurrence_kind
        except Exception:
            raise serializers.ValidationError("Occurrence Kind not found!")
    else:
        occurrence_kind = ""
    key_name = "RP_name_format"
    number_format = ""
    if key_name in instance.company.metadata:
        try:
            number_format = instance.company.metadata[key_name][
                occurrence_kind
            ]
        except Exception:
            if "default" in instance.company.metadata[key_name]:
                number_format = instance.company.metadata[key_name][
                    "default"
                ]
            else:
                raise serializers.ValidationError(
                    "Variáveis de nome inválidas!"
                )
    else:
        raise serializers.ValidationError("Variáveis de nome inválidas!")
    instance_type = number_format["type"]
    # Get datetime and serial arrays
    data = get_autonumber_array(instance.company.uuid, instance_type)
    # Get company prefix
    if "company_prefix" in instance.company.metadata:
        data["prefixo"] = instance.company.metadata["company_prefix"]
    else:
        data["prefixo"] = "[{}]".format(instance.company.name)
    # Make number
    try:
        number = number_format["format"].format(**data)
    except Exception as e:
        print(e)
        # Fallback
        # UHIT-RG-2018.0001
        number = "{prefixo}-{nome}-{anoCompleto}.{serialAno}".format(**data)
    instance.number = number

In [ ]:
def job_name_format(instance):
    if instance.number in [None, ""]:
        instance_type = "job"
        key_name = "{}_name_format".format(instance_type)
        # Get datetime and serial arrays
        data = get_autonumber_array(instance.company.uuid, instance_type)
        # Get company prefix
        if "company_prefix" in instance.company.metadata:
            data["prefixo"] = instance.company.metadata["company_prefix"]
        else:
            data["prefixo"] = "[{}]".format(instance.company.name)
        # Make number
        try:
            if key_name in instance.company.metadata:
                number = instance.company.metadata[key_name].format(**data)
            else:
                raise Exception("Variáveis de nome inválidas!")
        except Exception as e:
            print(e)
            # Fallback
            # UHIT-job-2018.0001
            number = "{prefixo}-{nome}-{anoCompleto}.{serialAno}".format(**data)

        instance.number = number

Reset the numbers of the reportings so that they agree to the chronological order

In [ ]:
Sequence.objects.filter(name__icontains=str(company.uuid)).update(last=0)

In [ ]:
Reporting.objects.filter(company=company).update(number="")

In [ ]:
while Reporting.objects.filter(company=company, number="").count():
    reportings = []
    for reporting in Reporting.objects.filter(company=company, number="").order_by('found_at')[:100]:
        add_reporting_number(reporting)
        reportings.append(reporting)
    bulk_update(reportings)

In [ ]:
jobs = []
for job in Job.objects.filter(company=company).order_by('start_date'):
    job_name_format(job)
    jobs.append(job)
bulk_update(jobs)

In [ ]:
from helpers.apps.job import total_and_executed_reporting

Archive 90% of the Job objects where all the reportings were already executed

In [ ]:
for job in Job.objects.filter(company=company):

    total, executed = total_and_executed_reporting(job)

    if total == executed and choice([False, True, True, True, True, True, True, True, True, True]):
        job.archived = True
        with DisableSignals():
            job.save()

## ReportingFile

Create 3000 new objects

Do a google image search for "bad road" and pick a random image, associate with a random Reporting

In [ ]:
available_files = []

for existing in tqdm(ReportingFile.upload.field.storage.listdir("images_mock")[0]):
    file_choices = ReportingFile.upload.field.storage.listdir("images_mock/{}".format(existing))[1]
    file_choices = [a for a in file_choices if ".jpg" in a]
    available_files += ["images_mock/{}/{}".format(existing, a) for a in file_choices]

In [ ]:
ReportingFile.objects.filter(reporting__company=company).delete()

In [ ]:
file_objects = []

reportings = (Reporting.objects.filter(company=company)
              .exclude(occurrence_type__occurrence_kind__in=["2"])
              .prefetch_related('created_by'))

for i in tqdm(range(1500)):
    reporting = choice(reportings)
    user = reporting.created_by
    file_path = choice(available_files)
    
    rf = ReportingFile(
        reporting=reporting,
        upload=file_path,
        created_by=user,
        include_rdo=True
    )
    file_objects.append(rf)
    
for reporting in tqdm(Reporting.objects.filter(company=company, occurrence_type__occurrence_kind="2")):
    user = reporting.created_by
    file_path = choice(available_files)
    
    rf = ReportingFile(
        reporting=reporting,
        upload=file_path,
        created_by=user,
        include_rdo=True
    )
    file_objects.append(rf)

result = ReportingFile.objects.bulk_create(file_objects, batch_size=100)
print(len(result))

## Contract

In [ ]:
wb = load_workbook(prefixo_folder + "contrato generico.xlsx")
sheetname = wb.sheetnames[0]
ws = wb[sheetname]

In [ ]:
header = []
values = []
good_fields = []
bad_fields = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)

In [ ]:
def get_connected_reference(company, resource, key, value):
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['name'] for a in select_options if a['value'].lower() == value.lower())

In [ ]:
created_by = user_firm_count['Gerente']['users'][0]

In [ ]:
Contract.objects.filter(firm__company=company).delete()

In [ ]:
for index, firm in enumerate(user_firm_count['Terceiro']['firms']):
    contract = Contract(
        firm = firm,
        name = "Contrato com fornecedor {}".format(firm.name),
        contract_start = datetime(2020, 6, 1),
        contract_end = datetime(2021, 5, 31),
        created_by = created_by,
        responsible = choice(user_firm_count['Sala técnica']['users']),
        firm_responsible = firm.manager,
        extra_info = {
            "r_c_number": "DEMO-{}".format(index),
            "accounting_classification": "CC-{}-XXX".format(index)
        }
    )
    contract.save()
    
    for resource in values:

        resource_kind = '1'

        additional_control = '1'

        amount = random.randint(10, 20)
        price = random.randint(10, 50) * 100
        
        created_resource = Resource.objects.create(
            company=company,
            name=resource["DESCRIÇÃO"],
            total_amount=amount,
            unit=resource["Unidade"],
            is_extra=True,
        )

        created_os_resource = ServiceOrderResource.objects.create(
            contract=contract,
            resource=created_resource,
            amount=amount,
            unit_price=price,
            created_by=created_by,
            resource_kind=resource_kind,
            provider='1',
            additional_control=additional_control,
        )


In [ ]:
months = ['7/2020','8/2020','9/2020','10/2020','11/2020','12/2020','1/2021','2/2021','3/2021','4/2021','5/2021', '6/2021']

In [ ]:
for contract in Contract.objects.filter(firm__company=company):
    total = sum([a[0] * a[1] for a in contract.resources.values_list('amount', 'unit_price')])
    spend_schedule = {
        a: total/12 for a in months
    }
    contract.spend_schedule = spend_schedule
    contract.save()

In [ ]:
ProcedureResource.objects.filter(reporting__company=company).delete()

In [ ]:
for reporting in Reporting.objects.filter(company=company):
    
    if reporting.status != status_executado and choice([0,1]):
        continue
    
    res_number = choice([1, 2])
    contract = Contract.objects.get(firm=reporting.firm)
    kind = get_connected_reference(company, 'reporting', "occurrenceKind", reporting.occurrence_type.occurrence_kind)
    resources = list(contract.resources.filter(resource__name__icontains=kind))
    if not len(resources):
        continue
    random.shuffle(resources)
    resources = resources[0:res_number]
    for resource in resources:
        usage = ProcedureResource(
            resource = resource.resource,
            service_order_resource = resource,
            created_by = created_by,
            amount = 1,
            unit_price = resource.unit_price,
            total_price = resource.unit_price,
            reporting = reporting,
            approval_status = "APPROVED_APPROVAL" if (
                reporting.status == status_executado and\
                reporting.executed_at < timezone.now().replace(day = timezone.now().day - 7) and\
                str(reporting.approval_step.uuid) in company.metadata['approved_approval_steps']
            ) else "WAITING_APPROVAL",
            approved_by = contract.responsible,
        )
        usage.save()
        usage.approval_date = reporting.executed_at + timedelta(days=1) if usage.approval_status == "APPROVED_APPROVAL" else None
        usage.creation_date = reporting.found_at
        usage.save()
        so_resource = usage.service_order_resource
        so_resource.remaining_amount -= 1
        so_resource.used_price += resource.unit_price
        so_resource.save()

In [ ]:
for contract in Contract.objects.filter(firm__company=company):
    for start, end in Arrow.span_range("month", contract.contract_start, contract.contract_end):
        resources = ProcedureResource.objects.filter(
            service_order_resource__contract=contract, 
            approval_date__gte=start._datetime, 
            approval_date__lte=end._datetime,
        )
        if resources.count():
            bulletin = MeasurementBulletin(
                firm = contract.firm,
                firm_manager = contract.firm_responsible,
                creation_date = end._datetime,
                measurement_date = end._datetime,
                created_by = contract.firm_responsible,
                contract = contract,
                approval_step = ApprovalStep.objects.get(
                    approval_flow__target_model="service_orders.MeasurementBulletin",
                    approval_flow__company=company,
                    name="Aprovado"
                ),
                editable = False
            )
            bulletin.save()
            resources.update(measurement_bulletin = bulletin)
            bulletin.total_price = sum([a[0] * a[1] for a in resources.values_list('unit_price', 'amount')])
            bulletin.save()